In [1]:
!pip install ibge
!pip install pyreadr
!pip install lerMicrodados
!pip install pandas
!pip install statsmodels




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.7/411.7 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lerMicrodados: filename=lerMicrodados-0.7-py3-none-any.whl size=14459 sha256=e1159f3b05d54f2414bf8703309615f347bbdaa8ac827c9fb1ff4a5b311fc1b1
  Stored in directory: /root/.cache/pip/wheels/a2/b3/00/df3c9957ced8c45ed638286e62399239d792ecfd20477fc372
Successfully built lerMicrodados


In [2]:
import ibge
import pandas as pd
import statsmodels.api as sm
import pandas as pd
import pyreadr
import numpy as np
from lerMicrodados import lerMicrodados as lM
import matplotlib.pyplot as plt
import pyreadr

In [3]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import pyreadr
import os

# Lista para armazenar os resultados de cada regressão
all_coeffs = []

# Loop pelos anos
for year in range(2012, 2025):  # De 2012 até 2025
    # Loop pelos trimestres
    for quarter in range(1, 5):  # De 1 até 4
        # Construir o nome base (que será usado como chave e parte do nome do arquivo)
        # Ex: PNADC_012012sel_mincer
        base_name_key = f"PNADC_0{quarter}{year}sel_mincer"

        # Nome completo do arquivo .Rdata
        # Ex: PNADC_012012sel_mincer.Rdata
        file_name_rdata = f"{base_name_key}.Rdata"
        file_path = file_name_rdata # Assume que os arquivos estão no mesmo diretório

        print(f"\nProcessando arquivo: {file_path}...")

        if not os.path.exists(file_path):
            print(f"Arquivo {file_path} não encontrado. Pulando.")
            continue

        try:
            # Carregar o arquivo .Rdata
            # A variável 'data_dict' será um dicionário retornado por pyreadr
            data_dict = pyreadr.read_r(file_path)

            # Acessar o DataFrame X_BR usando a chave base_name_key
            # Ex: X_BR = data_dict['PNADC_012012sel_mincer']
            if base_name_key in data_dict:
                X_BR_original = data_dict[base_name_key]
            else:
                print(f"  Chave '{base_name_key}' não encontrada no dicionário do arquivo {file_path}.")
                print(f"  Chaves disponíveis: {list(data_dict.keys())}. Pulando este arquivo.")
                continue

            X_BR = X_BR_original.copy() # Trabalhar com uma cópia

            # Verificar se a coluna 'UF' existe
            if 'UF' not in X_BR.columns:
                print(f"  Coluna 'UF' não encontrada no DataFrame X_BR do arquivo {file_path}. Pulando.")
                continue

            # Converter 'UF' para string para garantir a comparação correta
            X_BR['UF'] = X_BR['UF'].astype(str).str.strip()

            # Filtrar para Bahia (UF = '29') -> Este será o nosso 'df' (equivalente a X_BA)
            df = X_BR[X_BR['UF'] == '29'].copy() # .copy() é importante

            if df.empty:
                print(f"  Nenhum dado para a Bahia (UF '29') encontrado em {file_path} ou X_BR estava vazio para UF 29. Pulando.")
                continue

            print(f"  Dados da Bahia (UF '29') carregados. Número de observações inicial: {len(df)}")

            # --- A partir daqui, o código opera sobre 'df' (dados da Bahia) ---

            # 1. Colunas necessárias para os cálculos iniciais
            required_cols_initial = ['VD4016', 'VD3005', 'V2009', 'V4039']

            missing_cols = [col for col in required_cols_initial if col not in df.columns]
            if missing_cols:
                print(f"  Colunas ausentes em df (dados da Bahia) de {file_path}: {missing_cols}. Pulando.")
                continue

            # Remover linhas onde qualquer uma das colunas base é NaN
            df.dropna(subset=required_cols_initial, inplace=True)

            if df.empty:
                print(f"  DataFrame da Bahia vazio após o dropna inicial para {file_path}. Pulando.")
                continue

            # 2. Converter tipos de dados para as colunas relevantes
            df['VD3005'] = pd.to_numeric(df['VD3005'], errors='coerce') # Anos de estudo
            df['V2009'] = pd.to_numeric(df['V2009'], errors='coerce')   # Idade
            df['VD4016'] = pd.to_numeric(df['VD4016'], errors='coerce') # Rendimento mensal
            df['V4039'] = pd.to_numeric(df['V4039'], errors='coerce')   # Horas trabalhadas semanais

            # Remover NaNs que podem ter sido introduzidos pela conversão com 'coerce'
            df.dropna(subset=required_cols_initial, inplace=True)
            if df.empty:
                print(f"  DataFrame da Bahia vazio após conversão de tipos e segundo dropna para {file_path}. Pulando.")
                continue

            # 3. Engenharia de Variáveis
            df['experiencia'] = df['V2009'] - df['VD3005'] - 6
            df['experiencia^2'] = df['experiencia'] ** 2

            # Cálculo do ln(salário/hora)
            df['salario_hora_raw'] = np.where(df['V4039'] > 0, df['VD4016'] / df['V4039'], np.nan)
            df['ln_salario_hora'] = np.where(df['salario_hora_raw'] > 0, np.log(df['salario_hora_raw']), np.nan)

            # 4. Selecionar colunas para a regressão e remover NaNs de todas elas
            regression_cols = ['ln_salario_hora', 'VD3005', 'experiencia', 'experiencia^2']
            df_regression = df[regression_cols].copy() # Usar cópia para evitar SettingWithCopyWarning
            df_regression.dropna(inplace=True)

            # Verificar se há dados suficientes para a regressão
            min_obs_needed = len(regression_cols) -1 + 1 + 5 # preditores + intercepto + margem
            if df_regression.empty or len(df_regression) < min_obs_needed:
                print(f"  Dados insuficientes para regressão na Bahia em {file_path} ({len(df_regression)} observações válidas). Pulando.")
                continue

            # 5. Definir variáveis dependente (y) e independentes (X)
            y = df_regression['ln_salario_hora']
            X = df_regression[['VD3005', 'experiencia', 'experiencia^2']]
            X = sm.add_constant(X) # Adicionar a constante (intercepto)

            # 6. Estimar o modelo OLS
            model = sm.OLS(y, X).fit()
            coeffs = model.params

            # 7. Armazenar os resultados
            result = {
                'ano': year,
                'trimestre': quarter,
                'constante': coeffs.get('const', np.nan),
                'coef_educ (VD3005)': coeffs.get('VD3005', np.nan),
                'coef_exp': coeffs.get('experiencia', np.nan),
                'coef_exp^2': coeffs.get('experiencia^2', np.nan),
                'n_obs': model.nobs,      # Número de observações usadas no modelo
                'r_quadrado': model.rsquared # R²
            }
            all_coeffs.append(result)
            print(f"  SUCESSO: Regressão para Bahia {year} T{quarter}. R²: {model.rsquared:.4f}, N: {model.nobs}")

        except KeyError as e:
            # Este erro pode ocorrer se uma coluna esperada não existir em df_regression
            print(f"  Erro de chave (KeyError) ao processar {file_path}: {e}. Verifique nomes de colunas no DataFrame da Bahia. Pulando.")
        except Exception as e:
            print(f"  Ocorreu um erro inesperado ao processar {file_path}: {e}. Pulando.")

# Converter a lista de resultados em um DataFrame
results_df = pd.DataFrame(all_coeffs)

# Exibir os resultados
print("\n## Resultados Finais dos Coeficientes (Bahia - UF 29) ##")
if not results_df.empty:
    print(results_df)
    # Salvar os resultados em um arquivo CSV
    output_csv_file = "coeficientes_mincer_pnadc_BAHIA_2012_2025.csv"
    try:
        results_df.to_csv(output_csv_file, index=False, decimal=',', sep=';') # Formato CSV comum no Brasil
        print(f"\nResultados salvos em: {output_csv_file}")
    except Exception as e:
        print(f"Erro ao salvar o CSV: {e}")
else:
    print("Nenhum resultado foi gerado. Verifique os logs de erro e os arquivos de dados.")


Processando arquivo: PNADC_012012sel_mincer.Rdata...
Arquivo PNADC_012012sel_mincer.Rdata não encontrado. Pulando.

Processando arquivo: PNADC_022012sel_mincer.Rdata...
Arquivo PNADC_022012sel_mincer.Rdata não encontrado. Pulando.

Processando arquivo: PNADC_032012sel_mincer.Rdata...
Arquivo PNADC_032012sel_mincer.Rdata não encontrado. Pulando.

Processando arquivo: PNADC_042012sel_mincer.Rdata...
Arquivo PNADC_042012sel_mincer.Rdata não encontrado. Pulando.

Processando arquivo: PNADC_012013sel_mincer.Rdata...
Arquivo PNADC_012013sel_mincer.Rdata não encontrado. Pulando.

Processando arquivo: PNADC_022013sel_mincer.Rdata...
Arquivo PNADC_022013sel_mincer.Rdata não encontrado. Pulando.

Processando arquivo: PNADC_032013sel_mincer.Rdata...
Arquivo PNADC_032013sel_mincer.Rdata não encontrado. Pulando.

Processando arquivo: PNADC_042013sel_mincer.Rdata...
Arquivo PNADC_042013sel_mincer.Rdata não encontrado. Pulando.

Processando arquivo: PNADC_012014sel_mincer.Rdata...
Arquivo PNADC_0120